In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from matplotlib.pyplot import *
# Suppress warnings from pandas
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import f1_score, average_precision_score, precision_score, recall_score
from sklearn.tree import DecisionTreeClassifier
from IPython.display import Image as PImage
from subprocess import check_call
from PIL import Image, ImageDraw, ImageFont
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from  matplotlib.ticker import PercentFormatter
style.use('ggplot')


file_path = 'Consumer_Complaints.csv'
consumer_data = pd.read_csv(file_path,error_bad_lines=False, index_col=False, dtype='unicode')
#consumer_data=consumer_data.dropna()
consumer_data.Product=consumer_data.Product.str.strip().str.lower().str.replace(' ', '_').str.replace('-', '_')
consumer_data.Product=consumer_data.Product.str.strip().str.lower().str.replace(' ', '_').str.replace('-', '_')
consumer_data.Issue=consumer_data.Issue.str.strip().str.lower().str.replace(' ', '_').str.replace('-', '_')

consumer_data.columns =consumer_data.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('-', '_').str.replace('?', '')
consumer_data.sub_issue=consumer_data.sub_issue.str.strip().str.lower().str.replace(' ', '_').str.replace('-', '_')
consumer_data.sub_product=consumer_data.sub_product.str.strip().str.lower().str.replace(' ', '_').str.replace('-', '_')
consumer_data = consumer_data.drop_duplicates(consumer_data.columns, keep='last')
consumer_data['date_received'] =[datetime.strptime(x, '%m/%d/%Y') for x in consumer_data['date_received'] ]
consumer_data['date_sent_to_company'] =[datetime.strptime(x, '%m/%d/%Y') for x in consumer_data['date_sent_to_company'] ]


dispute_data=consumer_data[(consumer_data['consumer_disputed']=='Yes')|(consumer_data['consumer_disputed']=='No')]
#dispute_data.describe()
####count companies disputes and create dictionary
company_counts=dispute_data['company'].value_counts()
dfcompany_counts = pd.DataFrame({'counts':company_counts.values,'dic_company':company_counts.index,'companies':company_counts.index,})

dfcompany_counts['dic_company'] =["unique" if x==1 else dfcompany_counts['dic_company'].iloc[i] for i,x in enumerate(dfcompany_counts['counts'])]
dfcompany_counts['dic_company'] =["petite" if (x>1 and x<100) else dfcompany_counts['dic_company'].iloc[i] for i,x in enumerate(dfcompany_counts['counts'])]
dfcompany_counts['dic_company'] =['small' if (x>100 and x<1000)  else dfcompany_counts['dic_company'].iloc[i] for i,x in enumerate(dfcompany_counts['counts'])]
dfcompany_counts['dic_company'] =['medium' if (x>=1000 and x<5000) else dfcompany_counts['dic_company'].iloc[i] for i,x in enumerate(dfcompany_counts['counts'])]

company_dict = dict(zip(dfcompany_counts.companies, dfcompany_counts.dic_company))
 
dispute_data['company_code']=dispute_data['company'].map(company_dict)

#sns.catplot(y="company_code", hue="consumer_disputed", kind="count",data=dispute_data)
dispute_data=pd.get_dummies(data=dispute_data, columns=['product'])
dispute_data=pd.get_dummies(data=dispute_data, columns=['issue'])
#dispute_data=pd.get_dummies(data=dispute_data, columns=['sub_product'])
#dispute_data=pd.get_dummies(data=dispute_data, columns=['sub_issue'])
dispute_data=pd.get_dummies(data=dispute_data, columns=['company_public_response'])
dispute_data=pd.get_dummies(data=dispute_data, columns=['company_code'])
dispute_data=pd.get_dummies(data=dispute_data, columns=['consumer_consent_provided'])
dispute_data=pd.get_dummies(data=dispute_data, columns=['submitted_via'])

dispute_data=pd.get_dummies(data=dispute_data, columns=['company_response_to_consumer'])
dispute_data=pd.get_dummies(data=dispute_data, columns=['timely_response'])
dispute_data=pd.get_dummies(data=dispute_data, columns=['consumer_disputed'])

In [2]:
features=['product_bank_account_or_service',  'product_checking_or_savings_account',  'product_consumer_loan',  'product_credit_card',  'product_credit_reporting',  'product_debt_collection',  'product_money_transfers',  'product_mortgage',  'product_other_financial_service',  'product_payday_loan',  'product_prepaid_card',  'product_student_loan',  'product_virtual_currency',  'issue_account_opening,_closing,_or_management',  'issue_account_terms_and_changes',  'issue_adding_money',  'issue_advertising,_marketing_or_disclosures',  'issue_advertising_and_marketing',  'issue_application,_originator,_mortgage_broker',  'issue_application_processing_delay',  'issue_applied_for_loan/did_not_receive_money',  'issue_apr_or_interest_rate',  'issue_arbitration',  'issue_balance_transfer',  'issue_balance_transfer_fee',  'issue_bankruptcy',  'issue_billing_disputes',  'issue_billing_statement',  "issue_can't_contact_lender",  "issue_can't_repay_my_loan",  "issue_can't_stop_charges_to_bank_account",  'issue_cash_advance',  'issue_cash_advance_fee',  'issue_charged_bank_acct_wrong_day_or_amt',  "issue_charged_fees_or_interest_i_didn't_expect",  'issue_closing/cancelling_account',  'issue_collection_debt_dispute',  'issue_collection_practices',  'issue_communication_tactics',  "issue_cont'd_attempts_collect_debt_not_owed",  'issue_convenience_checks',  'issue_credit_card_protection_/_debt_protection',  'issue_credit_decision_/_underwriting',  'issue_credit_determination',  'issue_credit_line_increase/decrease',  'issue_credit_monitoring_or_identity_protection',  'issue_credit_reporting',  "issue_credit_reporting_company's_investigation",  'issue_customer_service/customer_relations',  'issue_customer_service_/_customer_relations',  'issue_dealing_with_my_lender_or_servicer',  'issue_delinquent_account',  'issue_deposits_and_withdrawals',  'issue_disclosure_verification_of_debt',  'issue_disclosures',  'issue_excessive_fees',  'issue_false_statements_or_representation',  'issue_fees',  'issue_forbearance_/_workout_plans',  'issue_fraud_or_scam',  'issue_getting_a_loan',  'issue_identity_theft_/_fraud_/_embezzlement',  'issue_improper_contact_or_sharing_of_info',  'issue_improper_use_of_my_credit_report',  'issue_incorrect/missing_disclosures_or_info',  'issue_incorrect_exchange_rate',  'issue_incorrect_information_on_credit_report',  'issue_late_fee',  'issue_lender_damaged_or_destroyed_property',  'issue_lender_damaged_or_destroyed_vehicle',  'issue_lender_repossessed_or_sold_the_vehicle',  'issue_lender_sold_the_property',  'issue_loan_modification,collection,foreclosure',  'issue_loan_servicing,_payments,_escrow_account',  'issue_lost_or_stolen_check',  'issue_lost_or_stolen_money_order',  'issue_making/receiving_payments,_sending_money',  'issue_managing,_opening,_or_closing_account',  'issue_managing_an_account',  'issue_managing_the_line_of_credit',  'issue_managing_the_loan_or_lease',  'issue_money_was_not_available_when_promised',  'issue_opening_an_account',  'issue_other',  'issue_other_fee',  'issue_other_service_issues',  'issue_other_transaction_issues',  'issue_overdraft,_savings_or_rewards_features',  'issue_overlimit_fee',  'issue_payment_to_acct_not_credited',  'issue_payoff_process',  'issue_privacy',  'issue_problems_caused_by_my_funds_being_low',  'issue_problems_when_you_are_unable_to_pay',  "issue_received_a_loan_i_didn't_apply_for",  'issue_repaying_your_loan',  'issue_rewards',  'issue_sale_of_account',  'issue_settlement_process_and_costs',  'issue_shopping_for_a_line_of_credit',  'issue_shopping_for_a_loan_or_lease',  'issue_struggling_to_pay_mortgage',  'issue_taking/threatening_an_illegal_action',  'issue_taking_out_the_loan_or_lease',  'issue_transaction_issue',  'issue_trouble_during_payment_process',  'issue_unable_to_get_credit_report/credit_score',  'issue_unauthorized_transactions/trans._issues',  'issue_unexpected/other_fees',  'issue_unsolicited_issuance_of_credit_card',  'issue_using_a_debit_or_atm_card',  'issue_wrong_amount_charged_or_received',  'company_public_response_Company believes complaint caused principally by actions of third party outside the control or direction of the company',  'company_public_response_Company believes complaint is the result of an isolated error',  'company_public_response_Company believes complaint relates to a discontinued policy or procedure',  'company_public_response_Company believes complaint represents an opportunity for improvement to better serve consumers',  'company_public_response_Company believes it acted appropriately as authorized by contract or law',  'company_public_response_Company believes the complaint is the result of a misunderstanding',  "company_public_response_Company can't verify or dispute the facts in the complaint",  'company_public_response_Company chooses not to provide a public response',  'company_public_response_Company disputes the facts presented in the complaint',  'company_public_response_Company has responded to the consumer and the CFPB and chooses not to provide a public response',  'company_code_AMERICAN EXPRESS COMPANY',  'company_code_Alliant Capital Management LLC',  'company_code_BANK OF AMERICA, NATIONAL ASSOCIATION',  'company_code_Blatt, Hasenmiller, Leibsker & Moore, LLC',  'company_code_CAPITAL ONE FINANCIAL CORPORATION',  'company_code_CITIBANK, N.A.',  'company_code_DISCOVER BANK',  'company_code_Ditech Financial LLC',  'company_code_ENCORE CAPITAL GROUP INC.',  'company_code_EQUIFAX, INC.',  'company_code_Experian Information Solutions Inc.',  'company_code_GOLDMAN SACHS BANK USA',  'company_code_HSBC NORTH AMERICA HOLDINGS INC.',  'company_code_JPMORGAN CHASE & CO.',  'company_code_LJ Ross Associates',  'company_code_NATIONSTAR MORTGAGE',  'company_code_Navient Solutions, LLC.',  'company_code_OCWEN LOAN SERVICING LLC',  'company_code_PNC Bank N.A.',  'company_code_PORTFOLIO RECOVERY ASSOCIATES INC',  'company_code_SELECT PORTFOLIO SERVICING, INC.',  'company_code_SUNTRUST BANKS, INC.',  'company_code_SYNCHRONY FINANCIAL',  'company_code_TD BANK US HOLDING COMPANY',  'company_code_TRANSUNION INTERMEDIATE HOLDINGS, INC.',  'company_code_U.S. BANCORP',  'company_code_WELLS FARGO & COMPANY',  'company_code_medium',  'company_code_petite',  'company_code_small',  'company_code_unique',  'consumer_consent_provided_Consent not provided',  'consumer_consent_provided_Consent provided',  'consumer_consent_provided_Consent withdrawn',  'consumer_consent_provided_Other',  'submitted_via_Email',  'submitted_via_Fax',  'submitted_via_Phone',  'submitted_via_Postal mail',  'submitted_via_Referral',  'submitted_via_Web',  'company_response_to_consumer_Closed',  'company_response_to_consumer_Closed with explanation',  'company_response_to_consumer_Closed with monetary relief',  'company_response_to_consumer_Closed with non-monetary relief',  'company_response_to_consumer_Closed with relief',  'company_response_to_consumer_Closed without relief',  'company_response_to_consumer_Untimely response',  'timely_response_Yes',]
y=dispute_data[ 'consumer_disputed_Yes']
#print(len(features),len(dispute_data.columns.tolist()))
X=dispute_data[features]
train_X, val_X, train_y, val_y = train_test_split(X, y,test_size=0.8, random_state = 0)

In [4]:
import eli5
from eli5.sklearn import PermutationImportance

logreg=LogisticRegression(solver='liblinear',class_weight={0:.29, 1:.71})
logreg_model =logreg.fit(train_X, train_y)
perm_logreg = PermutationImportance(logreg_model, random_state=1).fit(val_X, val_y)
log_insights=eli5.explain_weights_df(perm_logreg, feature_names = val_X.columns.tolist())
print(log_insights)

                                               feature    weight       std
0                                     product_mortgage  0.058477  0.000228
1       issue_loan_modification,collection,foreclosure  0.022556  0.000140
2       issue_loan_servicing,_payments,_escrow_account  0.019903  0.000119
3    company_response_to_consumer_Closed with expla...  0.004577  0.000152
4    company_response_to_consumer_Closed without re...  0.004247  0.000103
5       issue_application,_originator,_mortgage_broker  0.004085  0.000103
6                  company_response_to_consumer_Closed  0.004050  0.000043
7                                  company_code_medium  0.002749  0.000109
8                issue_disclosure_verification_of_debt  0.002250  0.000056
9                   issue_settlement_process_and_costs  0.002186  0.000132
10                          company_code_EQUIFAX, INC.  0.002117  0.000049
11               company_code_OCWEN LOAN SERVICING LLC  0.001970  0.000028
12                       

In [7]:
##dropping all the unnecesaary variables...##negative values of weight means better fittings when shuffling the values of the feature
log_insights=log_insights[(log_insights['weight']>=0)]
log_insights.shape

(92, 3)

In [9]:
best_log= LogisticRegression(C=0.0001, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l1', random_state=None,solver='liblinear', tol=0.0001, verbose=0, warm_start=False)
beslog_model=best_log.fit(train_X, train_y)
perm_beslog = PermutationImportance(beslog_model, random_state=1).fit(val_X, val_y)
beslog_insights=eli5.explain_weights_df(perm_beslog, feature_names = val_X.columns.tolist())
print(beslog_insights.shape)
beslog_insights=beslog_insights[(beslog_insights['weight']>=0)]
beslog_insights.shape

(171, 3)


(171, 3)

In [10]:
best_forest=RandomForestClassifier(bootstrap=True, class_weight={0: 0.4, 1: 0.6}, criterion='gini', max_depth=None, max_features=95, max_leaf_nodes=None, min_impurity_decrease=0.0,  min_impurity_split=None, min_samples_leaf=1,min_samples_split=2, min_weight_fraction_leaf=0.0,  n_estimators=90, n_jobs=None, oob_score=False, random_state=None, verbose=0, warm_start=False)
besfor_model=best_forest.fit(train_X, train_y)
perm_besfor = PermutationImportance(besfor_model, random_state=1).fit(val_X, val_y)
besfor_insights=eli5.explain_weights_df(perm_besfor, feature_names = val_X.columns.tolist())
print(besfor_insights.shape)
besfor_insights=besfor_insights[(besfor_insights['weight']>=0)]
besfor_insights.shape

(171, 3)


(39, 3)

In [11]:
besfor_insights

,feature,weight,std
0,consumer_consent_provided_Consent not provided,1.591053e-03,0.000109
1,consumer_consent_provided_Consent provided,1.173040e-03,0.000098
2,company_public_response_Company has responded ...,1.151244e-03,0.000090
3,company_code_Experian Information Solutions Inc.,4.170374e-04,0.000032
4,"company_code_TRANSUNION INTERMEDIATE HOLDINGS,...",3.493745e-04,0.000028
5,product_payday_loan,3.425432e-04,0.000013
6,company_public_response_Company believes it ac...,2.843141e-04,0.000035
7,product_money_transfers,2.556875e-04,0.000022
8,company_public_response_Company chooses not to...,2.312898e-04,0.000067
9,issue_credit_reporting_company's_investigation,1.929042e-04,0.000080


In [14]:
#besfor_insights.to_csv('Forest_tree_weights.csv')
log_insights.to_csv('logistic_classification_weights.csv')
beslog_insights.to_csv('best_log_weights.csv')